In [3]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq

from HARK.ConsumptionSaving.ConsGenIncProcessModel import (
    PersistentShockConsumerType,
    init_persistent_shocks
)

AiyagariAgentDictionary = {
    "CRRA": 5.0,  # Coefficient of relative risk aversion
    "Rfree": 1.03,  # Interest factor on assets
    "DiscFac": 0.96,  # Intertemporal discount factor
    "LivPrb": [1.0],  # Survival probability
    "AgentCount": 10000,  # Number of agents of this type (only matters for simulation)
    "aNrmInitMean": 0.0,  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd": 1.0,  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean": 0.0,  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd": 0.0,  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg": 1.0,  # Aggregate permanent income growth factor (only matters for simulation)
    "T_age": None,  # Age after which simulated agents are automatically killed
    "T_cycle": 1,  # Number of periods in the cycle for this agent type
    # Parameters for constructing the "assets above minimum" grid
    "aXtraMin": 0.001,  # Minimum end-of-period "assets above minimum" value
    "aXtraMax": 20,  # Maximum end-of-period "assets above minimum" value
    # Some other value of "assets above minimum" to add to the grid
    "aXtraExtra": np.array([0.005, 0.01]),
    "aXtraNestFac": 3,  # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount": 48,  # Number of points in the grid of "assets above minimum"
    # Parameters describing the income process
    "PermShkCount": 7,  # Number of points in discrete approximation to permanent income shocks
    "TranShkCount": 7,  # Number of points in discrete approximation to transitory income shocks
    "PermShkStd": [0.174],  # Standard deviation of log permanent income shocks
    "TranShkStd": [0],  # Standard deviation of log transitory income shocks
    "UnempPrb": 0.0,  # Probability of unemployment while working
    "UnempPrbRet": 0.0,  # Probability of "unemployment" while retired
    "IncUnemp": 0.0,  # Unemployment benefits replacement rate
    "IncUnempRet": 0.0,  # "Unemployment" benefits when retired
    "tax_rate": 0.0,  # Flat income tax rate
    "T_retire": 0,  # Period of retirement (0 --> no retirement)
    "BoroCnstArt": 0.0,  # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool": False,  # Use cubic spline interpolation when True, linear interpolation when False
    "vFuncBool": False,  # Whether to calculate the value function during solution
    "cycles": 0,
    "pLvlPctiles": np.concatenate(
        (
            [0.001, 0.005, 0.01, 0.03],
            np.linspace(0.05, 0.95, num=19),
            [0.97, 0.99, 0.995, 0.999],
        )
    ),
    "PermGroFac": [
        1.0
    ],
    "PrstIncCorr": 0.9
}

CapShare = 0.36
DeprRate = 0.08
Rfunc = lambda k : 1.0 + CapShare * k ** (CapShare - 1.0) - DeprRate
wFunc = lambda k: (1.0 - CapShare) * k ** (CapShare)
Rfunc_inv = lambda R : ((R - 1.0 + DeprRate)/CapShare)**(1.0/(CapShare - 1.0))

# Define our function of interest
def calc_Rfree_implied_by(Rfree_in):
    # Give the agents the proposed interest factor and calculate wage rate
    
    AiyagariType.Rfree = Rfree_in
    wRte = wFunc(Rfunc_inv(Rfree_in))
    
    # Solve and simulate the agents
    AiyagariType.solve()
    AiyagariType.initialize_sim()
    AiyagariType.simulate()
    
    # Extract the agents' level of assets (capital holdings) and productivity at the end
    pLvl_all = np.mean(np.sum(AiyagariType.history["pLvl"][300:500], axis=1))
    #cLvl_agg = np.mean(np.sum(AiyagariType.history["cLvl"][300:500], axis=1))
    aLvl_all = np.mean(np.sum(AiyagariType.history["aLvl"][300:500], axis=1))
    
    # Calculate aggregate capital and labor and take their ratio
    K = aLvl_all*wRte
    L = pLvl_all
    # L = AiyagariType.AgentCount
    k = K/L
    
    # Use the rule for determining Rfree to update the interest factor
    Rfree_out = Rfunc(k)
    return Rfree_out

def eqbm_func(R):
    calculated = calc_Rfree_implied_by(R)
    print("R", R, "diff", calculated - R)
    return calculated - R

sd = lambda rho, sigma : sigma*(1-rho**2)**0.5

sigma = 0.4
mu = [1, 3, 5]
rho = [0, 0.3, 0.6, 0.9]

output = []

for m in mu:
    for p in rho:
        AiyagariAgentDictionaryCopy = deepcopy(AiyagariAgentDictionary)
        AiyagariAgentDictionaryCopy["CRRA"] = m
        AiyagariAgentDictionaryCopy["PrstIncCorr"] = p
        AiyagariAgentDictionaryCopy["PermShkStd"] = [sd(p, sigma)]

        AiyagariType = PersistentShockConsumerType(**AiyagariAgentDictionaryCopy)
        AiyagariType.T_sim = 500
        AiyagariType.track_vars = ["aLvl", "cLvl", "pLvl"]

        AiyagariType.initialize_sim()
        AiyagariType.make_shock_history()

        R_star = brentq(eqbm_func, 0.96, 1.05)

        print("mu", m, "rho", p, "found R_star", R_star)

        pLvl_agg = np.mean(np.sum(AiyagariType.history["pLvl"][300:499], axis=1))
        cLvl_agg = np.mean(np.sum(AiyagariType.history["cLvl"][300:499], axis=1))
        aLvl_agg = np.mean(np.sum(AiyagariType.history["aLvl"][300:499], axis=1))

        print(pLvl_agg)

        income = (R_star - 1 + 0.08)*aLvl_agg + pLvl_agg

        output.append([1 - cLvl_agg/income, R_star])

        print(1 - cLvl_agg/income)

R 0.96 diff 0.48138605682607305
R 1.05 diff -0.12423580135651358
R 1.031537617952496 diff 0.19445095344648222
R 1.0428026854142725 diff -0.0746301919252973
R 1.034977744554921 diff 0.14992292960050135
R 1.0402020676443235 diff 0.02878357097244466
R 1.0409259081086488 diff -0.004236513146210763
R 1.0408330385801565 diff 0.00024091197483322446
R 1.040838035510514 diff 8.599233314310339e-07
R 1.04083805340707 diff -4.395439567872472e-11
R 1.0408380534060695 diff 4.126254893321857e-12
mu 1 rho 0 found R_star 1.0408380534060695
9999.57007441879
0.2392918925819627
R 0.96 diff 0.4757521848067925
R 1.05 diff -0.12427849028339055
R 1.0313591794722428 diff 0.14481125696549313
R 1.0413907792603834 diff -0.043370932606804025
R 1.037839397838511 diff 0.054503569795471885
R 1.0398170627098153 diff 0.007127790753841401
R 1.040039189558413 diff 0.00044248188955875456


KeyboardInterrupt: 

In [5]:
import pandas as pd

results = deepcopy(output)

for i in range(0, 12):
    results[i][0] = round(results[i][0]*100, 2)
    results[i][1] = round((results[i][1] - 1)*100, 4)

def rts(res):
    return "%.4f" % res[1] + "/" + "%.2f" % res[0]


reformatted = [
    [rts(results[0]), rts(results[4]), rts(results[8])], 
    [rts(results[1]), rts(results[5]), rts(results[9])], 
    [rts(results[2]), rts(results[6]), rts(results[10])], 
    [rts(results[3]), rts(results[7]), rts(results[11])]
]

row_names = {
    0:'ρ = 0',
    1:'ρ = 0.3',
    2:'ρ = 0.6',
    3:'ρ = 0.9'
}

df = pd.DataFrame(reformatted, columns=["µ = 1", "µ = 3", "µ = 5"])
df = df.rename(index = row_names)

print("HARK toolkit")
print("Net return to capital in % / aggregate saving rate in % (σ = 0.4)")
print(df)

aiyagari = [
    ["4.0649/23.87", "3.7816/24.44", "3.4177/25.22"],
    ["3.9554/24.09", "3.4188/25.22", "2.8032/26.66"],
    ["3.7567/24.50", "2.7835/26.71", "1.8070/29.37"],
    ["3.3054/25.47", "1.2894/31.00", "-0.3456/37.63"]
]

df = pd.DataFrame(aiyagari, columns=["µ = 1", "µ = 3", "µ = 5"])
df = df.rename(index = row_names)

print("\nAiyagari (1994)")
print("Net return to capital in % / aggregate saving rate in % (σ = 0.4)")
print(df)

HARK toolkit
Net return to capital in % / aggregate saving rate in % (σ = 0.4)
                µ = 1         µ = 3         µ = 5
ρ = 0    4.0838/23.93  3.8575/24.36  3.5693/24.95
ρ = 0.3  4.0054/24.02  3.5861/24.88  3.0908/25.98
ρ = 0.6  3.8580/24.29  3.0819/25.98  2.2328/28.14
ρ = 0.9  3.6054/24.81  2.1353/28.40  0.5484/33.67

Aiyagari (1994)
Net return to capital in % / aggregate saving rate in % (σ = 0.4)
                µ = 1         µ = 3          µ = 5
ρ = 0    4.0649/23.87  3.7816/24.44   3.4177/25.22
ρ = 0.3  3.9554/24.09  3.4188/25.22   2.8032/26.66
ρ = 0.6  3.7567/24.50  2.7835/26.71   1.8070/29.37
ρ = 0.9  3.3054/25.47  1.2894/31.00  -0.3456/37.63
